## TETRIS: Cálculo de la mejor posición dada una situación del tablero

En esta parte, tendremos una situación del tablero y una pieza que se está manejando en el momento, y el objetivo es conocer cuál es la mejor posición para situar la pieza. 
Para ello, primero se calculan todas las posiciones posibles, y luego se puntúa cada posición realizando un cálculo basado en cuatro métricas: El número de líneas que se completan, la altura máxima, la variación de las alturas de cada columna y el número de huecos, de forma que las tres últimas restan puntuación (por lo cual se quiere minimizarlas) y la primera suma puntuación (por lo cual, se quiere maximizar).

En primer lugar, importamos los módulos y clases necesarias:

In [12]:
import problema_espacio_estados as probee
import Piece as pieceClass
import Coordinate as coordinateClass
import Board as boardClass
import copy
import numpy
import math
import búsqueda_espacio_estados as búsqee
import operator
b_anchura = búsqee.BúsquedaEnAnchura(detallado=True)

A continuación, definimos una situación inicial del tablero junto con la pieza que se maneja:

In [13]:
piece = pieceClass.Piece([coordinateClass.Coordinate(1,1),coordinateClass.Coordinate(0,1),coordinateClass.Coordinate(1,0),coordinateClass.Coordinate(1,2)], "T")

board = boardClass.Board(20,10,piece)

#Llenamos el tablero con algunas piezas
board.matrix[19,1]=1
board.matrix[19,3]=1
board.matrix[19,4]=1
board.matrix[19,5]=1
board.matrix[19,6]=1
board.matrix[19,7]=1
board.matrix[19,8]=1

board.matrix[18,0]=1
board.matrix[18,1]=1
board.matrix[18,4]=1
board.matrix[18,5]=1
board.matrix[18,6]=1
board.matrix[18,7]=1
board.matrix[18,9]=1

a = board.printBoard()

 .  ■                .
 .■ ■ ■              .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .■ ■     ■ ■ ■ ■   ■.
 .  ■   ■ ■ ■ ■ ■ ■  .


Para calcular la mejor posición, simplemente llamamos al método calculateBestPosition() de la clase Board. Este método puede recibir un parámetro denominado 'detailed', con el cual puede elegirse si se muestran las distintas piezas y su puntuación mientras se realiza el cálculo. Se irán imprimiendo en pantalla las distintas posiciones, sus métricas y su puntuación total:

In [14]:
posiblePositions = board.calculatePosiblePositions()
sortedPositions = sorted(posiblePositions.items(), key=operator.itemgetter(1),reverse=True)
bestPosition = sortedPositions[0][0]

print("\nLa mejor posición es: ", bestPosition)


La mejor posición es:  (17,8), (17,9), (18,8), (17,7), 


A continuación, determinamos la secuencia de movimientos para alcanzar dicha posición:

In [15]:
# Creamos el tablero final
piece = bestPosition
finalBoard = boardClass.Board(20,10,piece)

#Llenamos el tablero con algunas piezas
finalBoard.matrix[19,1]=1
finalBoard.matrix[19,3]=1
finalBoard.matrix[19,4]=1
finalBoard.matrix[19,5]=1
finalBoard.matrix[19,6]=1
finalBoard.matrix[19,7]=1
finalBoard.matrix[19,8]=1

finalBoard.matrix[18,0]=1
finalBoard.matrix[18,1]=1
finalBoard.matrix[18,4]=1
finalBoard.matrix[18,5]=1
finalBoard.matrix[18,6]=1
finalBoard.matrix[18,7]=1
finalBoard.matrix[18,9]=1

a = finalBoard.printBoard()

 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .                   .
 .              ■ ■ ■.
 .■ ■     ■ ■ ■ ■ ■ ■.
 .  ■   ■ ■ ■ ■ ■ ■  .


Definimos las acciones y creamos el objeto ProblemaEspacioEstados:

In [16]:
def aplicabilidad(board):
    return board.canMoveDown()

def aplicación(board):
    updatedBoard = copy.deepcopy(board)
    updatedPiece = copy.deepcopy(updatedBoard.getCurrentPiece())
    updatedPiece.moveDown()
    updatedBoard.updateBoard(updatedPiece)
    return updatedBoard

moveDown = probee.Acción('Mover abajo', aplicabilidad, aplicación)

def aplicabilidad(board):
    return board.canMoveRight()

def aplicación(board):
    updatedBoard = copy.deepcopy(board)
    updatedPiece = copy.deepcopy(updatedBoard.getCurrentPiece())
    updatedPiece.moveRight()
    updatedBoard.updateBoard(updatedPiece)
    return updatedBoard

moveRight = probee.Acción('Mover a la derecha', aplicabilidad, aplicación)

def aplicabilidad(board):
    return board.canMoveLeft()

def aplicación(board):
    updatedBoard = copy.deepcopy(board)
    updatedPiece = copy.deepcopy(updatedBoard.getCurrentPiece())
    updatedPiece.moveLeft()
    updatedBoard.updateBoard(updatedPiece)
    return updatedBoard

moveLeft = probee.Acción('Mover a la izquierda', aplicabilidad, aplicación)

def aplicabilidad(board):
    return board.canRotateLeft()

def aplicación(board):
    updatedBoard = copy.deepcopy(board)
    updatedPiece = copy.deepcopy(updatedBoard.getCurrentPiece())
    updatedPiece.rotateLeft(board.width)
    updatedBoard.updateBoard(updatedPiece)
    return updatedBoard

rotateLeft = probee.Acción('Rotar a la izquierda', aplicabilidad, aplicación)

def aplicabilidad(board):
    return board.canRotateRight()

def aplicación(board):
    updatedBoard = copy.deepcopy(board)
    updatedPiece = copy.deepcopy(updatedBoard.getCurrentPiece())
    updatedPiece.rotateRight(board.width)
    updatedBoard.updateBoard(updatedPiece)
    return updatedBoard

rotateRight = probee.Acción('Rotar a la derecha', aplicabilidad, aplicación)

In [17]:
acciones = [moveDown,moveRight,moveLeft,rotateLeft,rotateRight]
estado_inicial = board
estado_final = finalBoard
tetris = probee.ProblemaEspacioEstados(
    acciones, estado_inicial, [estado_final])

Finalmente, determinamos el camino usando el algoritmo A*:

In [18]:
from scipy.spatial import distance

def h(nodo):
    estado = nodo.estado
    mini = 0
    for i in range(len(estado.currentPiece.coordinates)):
        a = estado.currentPiece.coordinates[i]
        b = estado_final.currentPiece.coordinates[i]
        x = b.getX()-a.getX()
        y = b.getY()-a.getY()
        mod = math.sqrt((x*x) + (y*y))
        mini += mod
    return mini
b_a_estrella = búsqee.BúsquedaAEstrella(h)

try:
    a = b_a_estrella.buscar(tetris)
    print(a)
except:
    print("No existe camino.")

['Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover abajo', 'Mover a la derecha', 'Mover abajo', 'Mover a la derecha', 'Mover abajo', 'Mover a la derecha', 'Mover abajo', 'Mover a la derecha', 'Mover abajo', 'Mover a la derecha', 'Mover abajo', 'Mover abajo', 'Mover a la derecha', 'Mover a la derecha', 'Rotar a la izquierda', 'Rotar a la izquierda']
